<a href="https://colab.research.google.com/github/DanielMalheiros/kaggle/blob/main/DanielMalheirosKaggleNaviFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NAVI A.I. New Ventures Bootcamp 07/21

# Daniel Malheiros
# Kaggle: Predict Future Sales


Challenge proposed by the **NAVI A.I bootcamp**. This project involves concepts such as feature engineering, tidy data mapping, random search grid search hyperparameter tunning and the usage of GPU empowered XGBoost.

Dataset from https://www.kaggle.com/c/competitive-data-science-predict-future-sales


**My GitHub:** https://github.com/DanielMalheiros

**My Linkedin:** https://www.linkedin.com/in/danieloliveiramalheiros/?locale=en_US

**NAVI A.I:** https://www.navi.com.ai/

Colab last updated on 28/07/2021

# Imports and functions

In [ ]:
from google.colab import files
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold

from xgboost import XGBRegressor

!pip install googletrans==4.0.0-rc1 --quiet
from googletrans import Translator


%matplotlib inline


     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 1.2 MB 14.3 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 65 kB 4.9 MB/s 
     |████████████████████████████████| 53 kB 3.0 MB/s 


In [ ]:
translator = Translator()
def translate(df, field):
    df[field] = df[field].apply(lambda value: translator.translate(value).text)
    return df  

def column_fix(x):
    try:
        return '{:0.0f}'.format(x)
    except:
        return x

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
           

# Understanding the problem and reading data

In this challenge, we must work with a time-series dataset consisting of daily sales data, provided by one of the largest Russian software firms - 1C Company. 

We need to predict total sales for every product and store in the next month. 

In [ ]:
#upload your kaggle.json

files.upload()

!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/ -f

!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [ ]:
!kaggle competitions download -c competitive-data-science-predict-future-sales --quiet
!echo "Ay" | unzip -q "*.zip"


4 archives were successfully processed.


In [ ]:
!ls | grep -v zip | grep csv

item_categories.csv
items.csv
sales_train.csv
sample_submission.csv
shops.csv
test.csv


In [ ]:
'''
sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
sample_submission.csv - a sample submission file in the correct format.
items.csv - supplemental information about the items/products.
item_categories.csv  - supplemental information about the items categories.
shops.csv- supplemental information about the shops.

*info from Kaggle's dataset description
'''

item_categories = pd.read_csv('item_categories.csv')
items = pd.read_csv('items.csv')
sales_train = pd.read_csv('sales_train.csv')
sample_submission = pd.read_csv('sample_submission.csv')
shops = pd.read_csv('shops.csv')
test = pd.read_csv('test.csv')

We have item.csv, and item_categories.csv, with the names of the items, their categories names, categories id, items id and item names.

In [ ]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [ ]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


shops.csv gives us the names of the shops and their id.

In [ ]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


sales_train.csv it's our timeseries. It is not in a good format yet. We will treat items, item categories and shops before merging them to sales_train. This dataframe is the bulk of our data. It is important to notice that we have several days with item_cnt (number of itens sold), but we must predict the sales of the next **month**, so, aggregating will be necessary in the near future.

In [ ]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


test.csv is our testing data. Notice that each unique tuple of item_id and shop_id is represented by a unique ID. Those IDs will be our identifiers, and we must predict the next month sales for each of them.

In [ ]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


And below, is the sample_submission.csv, the expected format of our predictions.

In [ ]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


So far, we know:

sales_train.csv is the bulk of our dataset, with plenty of data to analyze (more than 2kk rows). It's datatypes seems to be fine, with some exceptions as date as an object (not a datetime) and item_cnt_day (our target variable) being a float (it doesn't make sense to sell half an item, does it?)

items have more info about the products that the shops (shop_id variable of the dataframe above) sells (product name, product id and category id). There are 22169 different items.

item_categories is simply a csv with the relation of category id (item_category_id, of the dataframe above) and the name of the category. We will probably merge both of them in the near future. There are 83 different categories.

shops is similar to the dataframe item_categories, but for the shopkeepers. It has the relation of the shop_id variable from sales_train and the shop's name. There are 59 different shops.

And.....they all are in Cyrillic. We will translate them to seek insights in shops and products names.

# Preprocessing & feature engineering part 1: shops and itens.

Let's translate item category names and shop names to look for some insight. The functing used is declared above, in the **Imports and functions** section.


Translating shop names, replacing special char and checking for duplicates:

In [ ]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [ ]:
shops = translate(shops, 'shop_name')

In [ ]:
# treating special characters
shops['shop_name'] = shops['shop_name'].str.upper().replace('[#,@,&,!,?,"]', '')
shops['shop_name'] = shops['shop_name'].apply(lambda name: name.strip())

In [ ]:
shops.head()

,shop_name,shop_id
0,"! YAKUTSK ORDZHONIKIDZE, 56 FRAN",0
1,"! YAKUTSK SHOPPING CENTER ""CENTRAL"" FRAN",1
2,"ADYGEA SHOPPING CENTER ""MEGA""",2
3,"BALASHIHA TRK ""OCTOBER-KINOMIR""",3
4,"VOLZHSKY SHOPPING CENTER ""VOLGA MALL""",4


In [ ]:
# cgecking for duplicates
shops[['shop_name']].any().duplicated()

shop_name    False
dtype: bool

No duplicates in translated shop_name. Let's check for the shop names in which "SHOPPING" or "MALL" are in it. We can create a boolean column for those shops who are garanteed to be inside shopping centers. We will call this boolean column **in_shopping**.

In [ ]:
in_shopping = []

for idx, row in shops.iterrows():
    if ('SHOPPING' in row['shop_name']) | ('MALL' in row['shop_name']):
        in_shopping.append(1)
    else:
        in_shopping.append(0)
        
shops['in_shopping'] = in_shopping

We can create another feature using the translated shop names: let's search for shops with 'MOSCOW' in its name, meaning they are in the capital. Doing so, we will create the new boolean column **in_capital**.
       

In [ ]:
in_capital = []

for idx, row in shops.iterrows():
    if ('MOSCOW' in row['shop_name']):
        in_capital.append(1)
    else:
        in_capital.append(0)
        
shops['in_capital'] = in_shopping
       

Now we have two new boolean columns accounting for:

- if the shop is in a shopping center or mall
- if the shop is in the capital (Moscow)

We can now drop shop_name from the frame.





In [ ]:
shops = shops.drop(columns='shop_name')

Translating item category names and preprocessing them (to numerical values):

In [ ]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [ ]:
item_categories = translate(item_categories, 'item_category_name')

In [ ]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Headset / Headphones,0
1,Accessories - PS2.,1
2,Accessories - PS3,2
3,Accessories - PS4.,3
4,Accessories - PSP.,4


Looking inside the translated category names, we can see many of them are about similar stuff. Let's further aggregate them.

Example: ID 1 to 8 are different kind of acessories. Let's give them a unique ID for acessory. 

Then, we will use label_encoding to give them numeric values.

In [ ]:
category_names = list(item_categories['item_category_name'])

for idx in range(0,1):
    category_names[idx] = 'PC Headsets / Headphones'
for idx in range(1,8):
    category_names[idx] = 'Accessories'
category_names[8] = 'Tickets'
category_names[9] = 'Delivery'
for idx in range(10,18):
    category_names[idx] = 'Videogame Console'
for idx in range(18,25):
    category_names[idx] = 'Videogame Games'
category_names[25] = 'Accessories for games'
for idx in range(26,28):
    category_names[idx] = 'Phone games'
for idx in range(28,32):
    category_names[idx] = 'PC games'
for idx in range(32,37):
    category_names[idx] = 'Card'
for idx in range(37,42):
    category_names[idx] = 'Movie'
for idx in range(42,55):
    category_names[idx] = 'Books'
for idx in range(55,61):
    category_names[idx] = 'Music'
for idx in range(61,73):
    category_names[idx] = 'Gifts'
for idx in range(73,79):
    category_names[idx] = 'Software'
for idx in range(79,81):
    category_names[idx] = 'Office'
for idx in range(81,83):
    category_names[idx] = 'Clean'
category_names[83] = 'Power elements'

lb = preprocessing.LabelEncoder()
item_categories['item_categories_agg'] = lb.fit_transform(category_names)

In [ ]:
item_categories.head()

,item_category_name,item_category_id,item_categories_agg
0,PC - Headset / Headphones,0,10
1,Accessories - PS2.,1,0
2,Accessories - PS3,2,0
3,Accessories - PS4.,3,0
4,Accessories - PSP.,4,0


Since now we will use our new category, let's drop the old one:

In [ ]:
item_categories = item_categories.drop(columns='item_category_name')

Merging into items dataframe:

(We can drop item_name since it isn't of much help and item_category_id since now we have our own new category_ids.

In [ ]:
items = items.merge(item_categories, on='item_category_id')
items = items.drop(columns=['item_name','item_category_id'])

In [ ]:
items.head()

,item_id,item_categories_agg
0,0,7
1,2,7
2,3,7
3,4,7
4,5,7


In [ ]:
# checking for duplicates
items.duplicated().sum()

0

In [ ]:
items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22170 entries, 0 to 22169
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   item_id              22170 non-null  int64
 1   item_categories_agg  22170 non-null  int64
dtypes: int64(2)
memory usage: 519.6 KB


In [ ]:
shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   shop_id      60 non-null     int64
 1   in_shopping  60 non-null     int64
 2   in_capital   60 non-null     int64
dtypes: int64(3)
memory usage: 1.5 KB


Items and shops dataframes are ready.

# Preprocessing and feature engineering part 2: sales_train

In [ ]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


We can drop the date column. Why? Since we need aggregations on a monthly time window, and date_block_num (see in kaggle) represents each month in the dataframe, date isn't going to be useful to us.

In [ ]:
sales02 = sales_train.drop(columns='date').drop_duplicates()

Merging sales02 into items and shop:

In [ ]:
sales03 = sales02.merge(items, on='item_id')
sales04 = sales03.merge(shops, on='shop_id')
sales04.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_categories_agg,in_shopping,in_capital
0,0,59,22154,999.0,1.0,7,1,1
1,4,59,2573,249.0,1.0,8,1,1
2,0,59,2574,399.0,1.0,8,1,1
3,1,59,2574,399.0,1.0,8,1,1
4,2,59,2574,399.0,1.0,8,1,1


In [ ]:
# checking for missing values
sales04.isna().sum()

date_block_num         0
shop_id                0
item_id                0
item_price             0
item_cnt_day           0
item_categories_agg    0
in_shopping            0
in_capital             0
dtype: int64

Now, we have both shop_id and item_id in the sales dataframe. Let's join it with the test.csv frame so we can obtain the unique ID's that represent each tuple of shop_id and item_id.

We need to predict the item_cnt in the next month (not daily!) for each of those IDs. In our dataframe, there are more combinations of shop_id and item_id them in test.csv, meaning we won't need all of our data. Let's join our dataframe with the test dataframe to get the IDs we need.

In [ ]:
sales05 = pd.merge(sales04, test, how='right', on=['shop_id', 'item_id'])
sales05.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_categories_agg,in_shopping,in_capital,ID
0,20.0,5,5037,2599.0,1.0,17.0,0.0,0.0,0
1,22.0,5,5037,2599.0,1.0,17.0,0.0,0.0,0
2,23.0,5,5037,1999.0,1.0,17.0,0.0,0.0,0
3,24.0,5,5037,1999.0,1.0,17.0,0.0,0.0,0
4,28.0,5,5037,1299.0,1.0,17.0,0.0,0.0,0


In [ ]:
# checking for missing data
sales05.isna().sum()

date_block_num         102796
shop_id                     0
item_id                     0
item_price             102796
item_cnt_day           102796
item_categories_agg    102796
in_shopping            102796
in_capital             102796
ID                          0
dtype: int64

This operation left us with some missing values. Some shops don't sell some itens, or didn't sold an item in one of the days in the dataframe.

We can solve part of the missing data easily, because they are the features we created! 

In [ ]:
# getting in_shopping, in_capital and item categories again

shop_n_capital = sales04[['shop_id','in_shopping','in_capital']].drop_duplicates()
sales06 = sales05.drop(columns=['in_shopping','in_capital']).merge(shop_n_capital, how='left', on='shop_id')
sales07 = sales06.drop(columns='item_categories_agg').merge(items, how='left',on='item_id')
sales07.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID,in_shopping,in_capital,item_categories_agg
0,20.0,5,5037,2599.0,1.0,0,0,0,17
1,22.0,5,5037,2599.0,1.0,0,0,0,17
2,23.0,5,5037,1999.0,1.0,0,0,0,17
3,24.0,5,5037,1999.0,1.0,0,0,0,17
4,28.0,5,5037,1299.0,1.0,0,0,0,17


In [ ]:
# checking for missing data
sales07.isna().sum()

date_block_num         102796
shop_id                     0
item_id                     0
item_price             102796
item_cnt_day           102796
ID                          0
in_shopping                 0
in_capital                  0
item_categories_agg         0
dtype: int64

item_categories, in_shopping and in_capital are now full with valid data.

In [ ]:
# one-hot-encoding capital and shopping variables

capital = pd.get_dummies(sales07['in_capital'],prefix='capital', drop_first=True)
shoppping = pd.get_dummies(sales07['in_shopping'], prefix='shop', drop_first=True)


sales07 = sales07.join(capital)
sales07 = sales07.join(shoppping)

sales08 = sales07.drop(columns=['in_capital','in_shopping'])

We still have missing data for date_block_num, item_price and item_cnt_day. This may mean, like we said before, that some shoppings don't sell some item (by consequence, it doesn't have a price), or didn't sold it in some of the days. Let's use the function fillna to turn the missing data to zeroes.

**Author's note:** Before using fillna, I tried to impute the missing data using the algorithm **MissForest**;

MissForest is a machine learning-based imputation technique. It uses a Random Forest algorithm to do the task. Unfortunately, for this (and some) especific challenges, miss forest ends up lowering the RMSE (rooted mean squared error), so I dropped it in the end and play'd along with fillna.


You can read more on:
https://github.com/epsilon-machine/missingpy

In [ ]:
sales08 = sales08.fillna(0)
sales08.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID,item_categories_agg,capital_1,shop_1
0,20.0,5,5037,2599.0,1.0,0,17,0,0
1,22.0,5,5037,2599.0,1.0,0,17,0,0
2,23.0,5,5037,1999.0,1.0,0,17,0,0
3,24.0,5,5037,1999.0,1.0,0,17,0,0
4,28.0,5,5037,1299.0,1.0,0,17,0,0


In [ ]:
sales_imputed = sales08.copy()

We are ready for the next step!

# Preprocessing part 3: the pivot and tidy data

Now, we must aggregate our dataframe so we can get the item_cnt_MONTH, and not just day. There are many ways of doing so. We will stick with the **pivot()** method, but a group by would also work fine.

In [ ]:
sales_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 846163 entries, 0 to 846162
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   date_block_num       846163 non-null  float64
 1   shop_id              846163 non-null  int64  
 2   item_id              846163 non-null  int64  
 3   item_price           846163 non-null  float64
 4   item_cnt_day         846163 non-null  float64
 5   ID                   846163 non-null  int64  
 6   item_categories_agg  846163 non-null  int64  
 7   capital_1            846163 non-null  uint8  
 8   shop_1               846163 non-null  uint8  
dtypes: float64(3), int64(4), uint8(2)
memory usage: 93.3 MB


Passing date_block_num and item_cnt_day to int (again). Some of our operations turned it back to float.

In [ ]:
# passing item count and date_block_num to int (again)
sales_imputed[['date_block_num', 'item_cnt_day']] = sales_imputed[['date_block_num', 'item_cnt_day']].astype('int')

In [ ]:
sales_block_num = sales_imputed.pivot_table(values='item_cnt_day', index='ID', columns='date_block_num', aggfunc='sum', fill_value=0).clip(0,20)

We also clipped our sum values on 0 to 20 since the challenge itself declare that predictions can surpass 20 and can't be negative (see kaggle description of results).

In [ ]:
# reseting index
sales_block_num = sales_block_num.reset_index()

In [ ]:
# fixing column names
sales_block_num = sales_block_num.T.rename(column_fix).T

In [ ]:
sales_block_num.head()

date_block_num,ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,1,1,3,1,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,1,2,1
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
sales_block_num.describe()

date_block_num,ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
count,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000
mean,107099.500000,0.055401,0.062035,0.079823,0.057423,0.057712,0.067302,0.067162,0.077568,0.088497,0.086270,0.097960,0.142796,0.100271,0.110747,0.116886,0.095317,0.111144,0.114472,0.114767,0.139795,0.131569,0.138581,0.171317,0.253352,0.178492,0.162134,0.165542,0.152923,0.153838,0.153520,0.155593,0.184986,0.183903,0.179267
std,61834.358168,0.446544,0.527666,0.635657,0.422488,0.432118,0.549365,0.494631,0.542044,0.748609,0.671562,0.738845,0.861274,0.584106,0.716145,0.737723,0.544856,0.702236,0.633847,0.608652,0.716201,0.767685,0.746429,0.950292,1.053916,0.789128,0.744337,0.740491,0.792346,0.759261,0.671384,0.592061,0.712502,0.858740,0.762245
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,53549.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,107099.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,160649.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,214199.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000


Each column here besides ID now represents a month, and the value in it represents the item_cnt_month: how itens were sold that month. We are now ready to merge this pivoted table back to the data.

In [ ]:
df = sales_block_num.merge(sales_imputed, on='ID')
df = df.drop(columns=['date_block_num','item_cnt_day'])
df.head()

,ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,shop_id,item_id,item_price,item_categories_agg,capital_1,shop_1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,1,1,3,1,0,5,5037,2599.0,17,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,1,1,3,1,0,5,5037,2599.0,17,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,1,1,3,1,0,5,5037,1999.0,17,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,1,1,3,1,0,5,5037,1999.0,17,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,1,1,3,1,0,5,5037,1299.0,17,0,0


In [ ]:
# checking for missing data
df.isna().any().sum()

0

In [ ]:
# number of unique IDs in df
df['ID'].nunique()

214200

In [ ]:
# checking if we have the same unique IDs from test.csv
df['ID'].nunique() == test['ID'].nunique()

True

In [ ]:
len(df)

846163

That's not good! We have 214200 unique IDs, but 846163 rows. Let's treat this dataframe and make it **tidy**.

**Author's note:** "What is this **tidy data** you keeping talking about?" Tidy data is a standard way of mapping the meaning of a dataset to its structure. A dataset is messy or tidy depending on how rows, columns and tables are matched up with observations, variables and types. In tidy data:

- Every column is a variable.

- Every row is an observation.

- Every cell is a single value.

So, in our case, each ID must be a unique index, without duplicates.

You can read more in https://vita.had.co.nz/papers/tidy-data.html

In [ ]:
df_drop = df.drop_duplicates()

In [ ]:
in_shop = df.groupby(by='ID').max()[['shop_1']].reset_index().drop_duplicates()

In [ ]:
in_shop.head()

,ID,shop_1
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
in_capital = df_drop.groupby(by='ID').max()[['capital_1']].reset_index().drop_duplicates()

In [ ]:
in_capital.head()

,ID,capital_1
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
prefinal01 = df_drop.drop(columns=['capital_1','shop_1']).merge(in_shop, how='inner', on='ID')

In [ ]:
prefinal02 = prefinal01.merge(in_capital, how='inner', on='ID')

In [ ]:
tidy_data = prefinal02.drop(columns=['item_price']).drop_duplicates()

In [ ]:
tidy_data = tidy_data.drop(columns=['shop_id','item_id'])

In [ ]:
tidy_data

,ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,item_categories_agg,shop_1,capital_1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,1,1,3,1,0,17,0,0
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0
7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,1,2,1,17,0,0
9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,17,0,0
10,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351436,214195,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,8,0,0
351438,214196,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0
351439,214197,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,8,0,0
351440,214198,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0


Now we are talking! We have 214200 rows, one for each unique ID. 

# XGBoost hyperparameters tuning and training

Now, we must divide our data into train, validation and test.

We will look for hyperparameters and train our model with train set, use the validation to check its metric and them finally predict the test to submission.

We must predict the sales for month 34...but we don't have any data from month 34! How to solve this?

There is probably many many ways of doing this. Here, we are going with the following solution:

Let's drop month 33 and train our model with the target variable being the month 32. This will be our train set. Them, we will validate it on month 33, that will be fully new to our model.  This will be the validation set.

Then, finally, we will use all (almost all, I'll explain soon) our data to predict the month 34. This will be the test set.

In [ ]:
tidy_data.columns

Index(['ID', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       'item_categories_agg', 'shop_1', 'capital_1'],
      dtype='object')

In [ ]:
train_set = tidy_data.drop(columns=['33'])

X_train = train_set.drop(columns=['32']).values
y_train = train_set['32'].values

In [ ]:
validation_set = tidy_data.drop(columns=['0'])

X_validation = validation_set.drop(columns=['33']).values
y_validation = validation_set['33'].values

In [ ]:
test_set = tidy_data.drop(columns=['0','1'])

X_test = test_set.values

Why "almost" all data? It's becase we need X (train, test and validation) to have the same number of columns. Since we are removing 33 and 32 from X_train, we need to also remove two columns from X_validation and X_test.

In X_validation, we will remove columns 33 (target) and 0. In X_test, we will remove columns 0 and 1. Since they are the older months in the dataframe, we expect to have less impact then removing any other column.

Now, we will set the evaluation set. This set is used by the parameter **Early Stopping Rounds**.   The early stopping rounds parameter in XGBoost that takes an integer value which tells the algorithm when to stop if there’s no further improvement in the evaluation metric. It can prevent overfitting and improve the model’s performance. The evaluation set it use to calculate the early stopping metrics.

In [ ]:
eval_set = [(X_train, y_train), (X_validation, y_validation)]

Let's define our grid of hyperparamers. Our random search will look for combinations of this parameters when looking for the best combination for our model.

In [ ]:
param_grid = {
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}

Setting our model:

In [ ]:
xgb = XGBRegressor(eval_metric= 'rmse',
                   tree_method='gpu_hist',
                   objective='reg:squarederror',
                   early_stopping_rounds=10,
                   eval_set=eval_set,
                   verbosity=2
                   )

Our evaluaution metric will be root mean squared error, the metric used for ranks in most Kaggle competitions. The secret here is the parameter **tree_method**. When setting our tree_method as 'gpu_hist', we can use the power of Google's Colab GPUs to enhance our model fit. If you have a good GPU in your computer, you can also set use this parameter (and configure your GPU) to hasten your models.

Early stopping rounds will be set as 10. This means that if after 10 epochs the metric (in this case, RMSE) don't present improvements, the model stops the fitting.

Since we have a fairly big dataframe and are using a lot of different parameters on our param_grid, let's set folds as 10 and param_combinations (param_comb, which will be passed as n_iter on random search) as 100 (Meaning the model will test 100 different combinations of the parameters). More iterations (n_iter/param_comb) will cover a wider search space and more cv folds (folds) reduces the chances of overfitting, but raising each will increase the run time.

In [ ]:
folds = 10
param_comb = 100

kf = KFold(n_splits=folds,
            shuffle = True,
            random_state = 42)

random_search = RandomizedSearchCV(xgb,
                                  scoring='neg_mean_squared_error',
                                  param_distributions=param_grid,
                                  n_iter=param_comb,
                                  cv=kf.split(X_train,y_train),
                                  verbose=2,
                                  random_state=42)

In [ ]:
random_search.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] subsample=0.7, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  subsample=0.7, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4, total=   2.0s
[CV] subsample=0.7, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV]  subsample=0.7, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4, total=   0.9s
[CV] subsample=0.7, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4 
[CV]  subsample=0.7, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4, total=   0.9s
[CV] subsample=0.7, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4 
[CV]  subsample=0.7, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4, total=   0.9s
[CV] subsample=0.7, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed: 124.2min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x7f9a89d724d0>,
                   error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1,
                                          early_stopping_rounds=10,
                                          eval_metric='rmse',
                                          eval_set=[(array([[     0,      0,      0, ...,     17,      0,      0],
       [     1,      0,      0, ...,      8,      0,      0],
       [     2,      0,      0, ...,     17,      0,      0],
       ...,
       [214197,      1,      0, ...,      8,      0,      0],
       [214...
                                        'gamma': [0, 0.25, 0.5, 1.0],
                                        'learning_rate': [0.001, 0.01, 0.1, 0.2,
               

In [ ]:
print('\nBest estimator:')
print(random_search.best_estimator_)
print('\nBest score (negative MSE) for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\nBest hyperparameters:')
print(random_search.best_params_)


Best estimator:
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
             colsample_bynode=1, colsample_bytree=0.5, early_stopping_rounds=10,
             eval_metric='rmse',
             eval_set=[(array([[     0,      0,      0, ...,     17,      0,      0],
       [     1,      0,      0, ...,      8,      0,      0],
       [     2,      0,      0, ...,     17,      0,      0],
       ...,
       [214197,      1,      0, ...,      8,      0,      0],
       [214198,      0,      0, ...,      7,      0,      0],
       [214199,      0,      0, ...,      7,      0,      0]]),
                        array([1, 0, 2, ..., 0, 0, 0], dtype=int64)),
                       (array([[     0,      0...
                        array([0, 0, 1, ..., 0, 0, 0], dtype=int64))],
             gamma=0.25, importance_type='gain', learning_rate=0.1,
             max_delta_step=0, max_depth=15, min_child_weight=7.0, missing=None,
             n_estimators=100, n_jobs=1, nthread=

In [ ]:
train_predict = random_search.predict(X_train).clip(0,20)

print(f"The TRAIN RMSE was: {rmse(train_predict, y_train)}")

The TRAIN RMSE was: 0.6523205988639317


In [ ]:
validation_predict = random_search.predict(X_validation).clip(0,20)

print(f"The TEST RMSE was: {rmse(validation_predict, y_validation)}")

The TEST RMSE was: 0.6751669916316173


In [ ]:
submission = random_search.predict(X_test).clip(0,20)

In [ ]:
submission_dataframe = pd.DataFrame()
submission_dataframe['ID'] = tidy_data['ID']
submission_dataframe['item_cnt_month'] = submission
submission_dataframe['ID'] = submission_dataframe['ID'].astype('int')


In [ ]:
submission_dataframe

,ID,item_cnt_month
0,0,0.721944
6,1,0.100239
7,2,0.613901
9,3,0.539680
10,4,0.653903
...,...,...
351436,214195,0.044962
351438,214196,0.004678
351439,214197,0.034307
351440,214198,0.014356


In [ ]:
# saving imputed dataframe to localfiles
submission_dataframe.to_csv('submission_dataframe_2.csv', index=False)
files.download('submission_dataframe_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The submission from this process got ranked as 6541 at Kaggle, with a RMSE of 1.05059! At the time being (July/2021), ranked at 6556.